In [ ]:
import numpy as np 
import pandas as pd 
from pathlib import Path
from fastai.text import *

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
path=Path('/content/gdrive/My Drive/covidnlp')

In [ ]:
df= pd.read_csv(path/'train.csv', encoding='utf-8')

In [ ]:
import os
for dirname, _, filenames in os.walk('/content/gdrive/My Drive/covidnlp'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/content/gdrive/My Drive/covidnlp/train.csv
/content/gdrive/My Drive/covidnlp/test-sample.csv
/content/gdrive/My Drive/covidnlp/databunch.pkl
/content/gdrive/My Drive/covidnlp/filename.pth.pth
/content/gdrive/My Drive/covidnlp/filename2.pth.pth
/content/gdrive/My Drive/covidnlp/train-data/00068/00068-08.txt
/content/gdrive/My Drive/covidnlp/train-data/00068/00068-03.txt
/content/gdrive/My Drive/covidnlp/train-data/00068/00068-22.txt
/content/gdrive/My Drive/covidnlp/train-data/00068/00068-10.txt
/content/gdrive/My Drive/covidnlp/train-data/00068/00068-24.txt
/content/gdrive/My Drive/covidnlp/train-data/00068/00068-07.txt
/content/gdrive/My Drive/covidnlp/train-data/00068/00068-02.txt
/content/gdrive/My Drive/covidnlp/train-data/00068/00068-15.txt
/content/gdrive/My Drive/covidnlp/train-data/00068/00068-13.txt
/content/gdrive/My Drive/covidnlp/train-data/00068/00068-20.txt
/content/gdrive/My Drive/covidnlp/train-data/00068/00068-18.txt
/content/gdrive/My Drive/covidnlp/train-data/00068/

In [ ]:
texts = []
for i in range(len(df['Filename'])):
    fn = df['Filename'][i]
    folder = fn.split("-")[0]
    f = open('/content/gdrive/My Drive/covidnlp/train-data/'+folder+'/'+fn, "r")
    texts.append(f.read())

df['text'] = texts

In [ ]:
df

,Filename,Blinding of intervention,Blinding of Outcome assessment,Classes,text
0,00060-02.txt,P,P,PP,A Multicomponent Intervention To Prevent Major...
1,00060-03.txt,N,N,NN,Original article\nSelf-management versus conve...
2,00060-04.txt,N,N,NN,Original Research\nHome Management of Oral Ant...
3,00060-05.txt,P,P,PP,For personal use only. Not to be reproduced wi...
4,00060-06.txt,P,P,PP,© 2016 Pozzi et al. This work is published and...
...,...,...,...,...,...
623,00458-20.txt,N,N,NN,﻿Effect of daily versus twice weekly long-term...
624,00458-21.txt,P,P,PP,﻿Eur J Nutr 39 : 263–269 (2000)\n© Steinkopff ...
625,00458-22.txt,N,N,NN,﻿ARTICLE\n \n\n \nWeekly vs Daily Iron and Fol...
626,00458-24.txt,N,N,NN,﻿Symposium: Improving Adolescent Iron Status\n...


In [ ]:
from functools import partial
import io
import os

df['text'] = df['text'].str.replace("[^a-zA-Z]", " ")

In [ ]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords 
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# tokenization 
tokenized_doc = df['text'].apply(lambda x: x.split())

# remove stop-words 
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

# de-tokenization 
detokenized_doc = [] 
for i in range(len(df)): 
    t = ' '.join(tokenized_doc[i]) 
    detokenized_doc.append(t) 

df['text'] = detokenized_doc

In [ ]:
df

,Filename,Blinding of intervention,Blinding of Outcome assessment,Classes,text
0,00060-02.txt,P,P,PP,A Multicomponent Intervention To Prevent Major...
1,00060-03.txt,N,N,NN,Original article Self management versus conven...
2,00060-04.txt,N,N,NN,Original Research Home Management Oral Anticoa...
3,00060-05.txt,P,P,PP,For personal use Not reproduced without permis...
4,00060-06.txt,P,P,PP,Pozzi et al This work published licensed Dove ...
...,...,...,...,...,...
623,00458-20.txt,N,N,NN,Effect daily versus twice weekly long term iro...
624,00458-21.txt,P,P,PP,Eur J Nutr Steinkopff Verlag Summary Backgroun...
625,00458-22.txt,N,N,NN,ARTICLE Weekly vs Daily Iron Folic Acid Supple...
626,00458-24.txt,N,N,NN,Symposium Improving Adolescent Iron Status Chi...


In [ ]:
df = df.iloc[np.random.permutation(len(df))]
cut1 = int(0.8 * len(df)) + 1
df_train, df_valid = df[:cut1], df[cut1:]

In [ ]:
data_lm = TextLMDataBunch.from_df(path=path,
 train_df=df_train,
 valid_df=df_valid, 
 label_cols='Blinding of intervention', 
 text_cols='text',
 bs=16)

In [ ]:
#data_lm.save('/content/gdrive/My Drive/covidnlp/','databunch.pkl')

In [ ]:
lm = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3)

In [ ]:
#lm.lr_find()

In [ ]:
#lm.recorder.plot(suggestion=True)

In [ ]:
lm.fit_one_cycle(1)

epoch,train_loss,valid_loss,accuracy,time
0,5.318124,4.960936,0.273988,02:12


In [ ]:
lm.save_encoder(path/'filename.pth')

In [ ]:
data_clas=TextClasDataBunch.from_df(path=path,train_df=df_train, 
                                    valid_df=df_valid, 
                                    vocab=data_lm.train_ds.vocab, 
                                    label_cols='Blinding of intervention', 
                                    text_cols='text',
                                    bs=64)

In [ ]:
clas = text_classifier_learner(data_clas,AWD_LSTM,drop_mult=0.7)

In [ ]:
clas.load_encoder(path/'filename.pth')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (503 items)
x: TextList
xxbos xxmaj original research article xxmaj two randomized controlled trials comparing xxmaj hulka xxmaj filshie xxmaj clips tubal sterilization xxmaj xxunk xxmaj xxunk xxmaj deborah xxmaj xxunk xxmaj david xxmaj xxunk xxmaj milton xxmaj xxunk xxmaj jorge xxmaj lasso de la xxmaj xxunk xxmaj arturo xxmaj remes xxmaj xxunk xxmaj john xxmaj xxunk xxmaj david xxmaj limb xxmaj serge xxmaj xxunk xxmaj roberto xxmaj santiso xxmaj xxunk xxmaj luis xxmaj xxunk xxmaj itic xxmaj xxunk xxmaj family xxmaj health xxmaj international xxmaj research xxmaj triangle xxmaj park xxup nc xxup usa b xxmaj the xxmaj clinical xxmaj investigator xxmaj team xxmaj received xxmaj june accepted xxmaj september xxmaj abstract xxmaj to compare effectiveness safety xxmaj filshie xxmaj clip xxmaj system xxmaj hulka xxmaj clip xxmaj system applied via minilaparotomy laparoscopy conducted multicenter randomized controlled trials women minilapar

In [ ]:
#clas.lr_find()

In [ ]:
#clas.recorder.plot(suggestion=True)

In [ ]:
clas.fit_one_cycle(1)

epoch,train_loss,valid_loss,accuracy,time
0,1.162545,1.054188,0.504000,01:08


In [ ]:
clas.unfreeze()

In [ ]:
clas.fit_one_cycle(10)

epoch,train_loss,valid_loss,accuracy,time
0,1.012465,1.018925,0.528000,01:18
1,1.010737,0.970368,0.560000,01:20
2,0.958278,0.915146,0.568000,01:17
3,0.916255,0.879932,0.544000,01:17
4,0.871967,0.863476,0.576000,01:18
5,0.829018,0.924641,0.592000,01:19
6,0.787010,0.950342,0.536000,01:19
7,0.757835,0.945803,0.560000,01:20
8,0.734734,0.943507,0.544000,01:18
9,0.711901,0.935932,0.560000,01:19


In [ ]:
df= pd.read_csv(path/'test.csv', encoding='utf-8')

In [ ]:
import os
for dirname, _, filenames in os.walk('/content/gdrive/My Drive/covidnlp'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/content/gdrive/My Drive/covidnlp/train.csv
/content/gdrive/My Drive/covidnlp/test-sample.csv
/content/gdrive/My Drive/covidnlp/databunch.pkl
/content/gdrive/My Drive/covidnlp/filename.pth.pth
/content/gdrive/My Drive/covidnlp/filename2.pth.pth
/content/gdrive/My Drive/covidnlp/test.csv
/content/gdrive/My Drive/covidnlp/train-data/00068/00068-08.txt
/content/gdrive/My Drive/covidnlp/train-data/00068/00068-03.txt
/content/gdrive/My Drive/covidnlp/train-data/00068/00068-22.txt
/content/gdrive/My Drive/covidnlp/train-data/00068/00068-10.txt
/content/gdrive/My Drive/covidnlp/train-data/00068/00068-24.txt
/content/gdrive/My Drive/covidnlp/train-data/00068/00068-07.txt
/content/gdrive/My Drive/covidnlp/train-data/00068/00068-02.txt
/content/gdrive/My Drive/covidnlp/train-data/00068/00068-15.txt
/content/gdrive/My Drive/covidnlp/train-data/00068/00068-13.txt
/content/gdrive/My Drive/covidnlp/train-data/00068/00068-20.txt
/content/gdrive/My Drive/covidnlp/train-data/00068/00068-18.txt
/content

In [ ]:
texts = []
for i in range(len(df['Id'])):
    fn = df['Id'][i]
    folder = fn.split("-")[0]
    f = open('/content/gdrive/My Drive/covidnlp/test-data/test-data/'+folder+'/'+fn, "r", encoding="utf8", errors='ignore')
    texts.append(f.read())

df['text'] = texts

In [ ]:
df

,Id,Prediction,text
0,00002-02.txt,NaN,﻿Abstract\nObjective: To assess the effect of ...
1,00002-04.txt,NaN,﻿Abstract\nBACKGROUND: Steroid pre-treatments ...
2,00002-06.txt,NaN,﻿effect accross all trials. The pooled results...
3,00002-07.txt,NaN,"﻿Human Reproduction vol.11 no.5 pp. 1035-1037,..."
4,00002-08.txt,NaN,﻿576\nProgestogen therapy during pituitary des...
...,...,...,...
390,00053-26.txt,NaN,ORIGINAL ARTICLE\nNottingham trial of faecal o...
391,00053-27.txt,NaN,Baseline Findings of the Italian Multicenter\n...
392,00053-28.txt,NaN,"1310 Articles | JNCI Vol. 103, Issue 17 | Sept..."
393,00053-29.txt,NaN,Acceptability of flexible sigmoidoscopy screen...


In [ ]:
from functools import partial
import io
import os

df['text'] = df['text'].str.replace("[^a-zA-Z]", " ")
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords 
stop_words = stopwords.words('english')
# tokenization 
tokenized_doc = df['text'].apply(lambda x: x.split())

# remove stop-words 
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

# de-tokenization 
detokenized_doc = [] 
for i in range(len(df)): 
    t = ' '.join(tokenized_doc[i]) 
    detokenized_doc.append(t) 

df['text'] = detokenized_doc

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df

,Id,Prediction,text
0,00002-02.txt,NaN,Abstract Objective To assess effect pretreatme...
1,00002-04.txt,NaN,Abstract BACKGROUND Steroid pre treatments may...
2,00002-06.txt,NaN,effect accross trials The pooled results odds ...
3,00002-07.txt,NaN,Human Reproduction vol pp A combination noreth...
4,00002-08.txt,NaN,Progestogen therapy pituitary desensitization ...
...,...,...,...
390,00053-26.txt,NaN,ORIGINAL ARTICLE Nottingham trial faecal occul...
391,00053-27.txt,NaN,Baseline Findings Italian Multicenter Randomiz...
392,00053-28.txt,NaN,Articles JNCI Vol Issue September DOI jnci djr...
393,00053-29.txt,NaN,Acceptability flexible sigmoidoscopy screening...


In [ ]:
for i in df.text:
  predicted = clas.predict(i)
  df['Prediction'] = str(predicted[0])+str(predicted[0])


In [ ]:
df

,Id,Prediction,text
0,00002-02.txt,NN,Abstract Objective To assess effect pretreatme...
1,00002-04.txt,NN,Abstract BACKGROUND Steroid pre treatments may...
2,00002-06.txt,NN,effect accross trials The pooled results odds ...
3,00002-07.txt,NN,Human Reproduction vol pp A combination noreth...
4,00002-08.txt,NN,Progestogen therapy pituitary desensitization ...
...,...,...,...
390,00053-26.txt,NN,ORIGINAL ARTICLE Nottingham trial faecal occul...
391,00053-27.txt,NN,Baseline Findings Italian Multicenter Randomiz...
392,00053-28.txt,NN,Articles JNCI Vol Issue September DOI jnci djr...
393,00053-29.txt,NN,Acceptability flexible sigmoidoscopy screening...


In [ ]:
finalpredict = df[['Id', 'Prediction']]
finalpredict

,Id,Prediction
0,00002-02.txt,NN
1,00002-04.txt,NN
2,00002-06.txt,NN
3,00002-07.txt,NN
4,00002-08.txt,NN
...,...,...
390,00053-26.txt,NN
391,00053-27.txt,NN
392,00053-28.txt,NN
393,00053-29.txt,NN


In [ ]:
finalpredict.to_csv('text.csv', index=False)